In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [2]:
```bash
   python dataset_prep.py

SyntaxError: invalid syntax (3110722330.py, line 1)

In [ ]:
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32
EPOCHS = 30
NUM_CLASSES = 59
DATA_DIR = "data_split"

In [ ]:
def prepare_data():
    train_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

    val_test_gen = ImageDataGenerator(rescale=1./255)

    train_set = train_gen.flow_from_directory(
        os.path.join(DATA_DIR, "train"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical"
    )

    val_set = val_test_gen.flow_from_directory(
        os.path.join(DATA_DIR, "val"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical"
    )

    test_set = val_test_gen.flow_from_directory(
        os.path.join(DATA_DIR, "test"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )

    return train_set, val_set, test_set

In [ ]:
def build_resnet(num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False,
                          input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
def plot_history(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def main():
    train_set, val_set, test_set = prepare_data()
    model = build_resnet(NUM_CLASSES)

    early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

    history = model.fit(train_set,
                        validation_data=val_set,
                        epochs=EPOCHS,
                        callbacks=[early_stop])

In [ ]:
 test_loss, test_accuracy = model.evaluate(test_set)
    print(f"Test accuracy: {test_accuracy:.2f}")

    # Classification report
    y_true = test_set.classes
    y_pred = np.argmax(model.predict(test_set), axis=1)
    print(classification_report(y_true, y_pred, target_names=list(test_set.class_indices.keys())))

    # Save model
    # model.save("resnet50_model.keras")

    # Plot training curves
    plot_history(history)


if __name__ == "__main__":
    main()